In [ ]:
%%configure -f
{
"conf": {
     "spark.dynamicAllocation.disableIfMinMaxNotSpecified.enabled": true,
   }
}

In [ ]:
# Placeholders for input parameters
anomaly_found = False
anomaly_time = ''
anomaly_location = ''
blob_account_name = ''
azure_storage_domain = ''
input_ais_csv_file = ''
input_image = ''
input_image_low_res = ''
ship_bb_image_low_res = '' 
ship_bb_image_high_res = ''
ais_image = ''
anomaly_image = ''


In [ ]:
# Initiate logging
import logging
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.azure.trace_exporter import AzureExporter
from opencensus.trace import config_integration
from opencensus.trace.samplers import AlwaysOnSampler
from opencensus.trace.tracer import Tracer

config_integration.trace_integrations(['logging'])

instrumentation_connection_string = mssparkutils.credentials.getSecretWithLS("keyvault", "AppInsightsConnectionString")

logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(connection_string=instrumentation_connection_string))
logger.setLevel(logging.INFO)

tracer = Tracer(
    exporter=AzureExporter(
        connection_string=instrumentation_connection_string
    ),
    sampler=AlwaysOnSampler()
)

# Spool parameters
run_time_parameters = {'custom_dimensions': {
    'anomaly_found': anomaly_found,
    'anomaly_location': anomaly_location,
    'anomaly_time': anomaly_time,
    'blob_account_name': blob_account_name, 
    'notebook_name': mssparkutils.runtime.context['notebookname']
} }
  
logger.info(f"{mssparkutils.runtime.context['notebookname']}: INITIALISED", extra=run_time_parameters)

In [ ]:
from pyspark.sql import SparkSession
import json
from azure.identity import ClientSecretCredential
from azure.eventgrid import EventGridPublisherClient, EventGridEvent
from datetime import datetime
from urllib.parse import quote 


In [ ]:
# Get config values 
sc = spark.sparkContext
spark = SparkSession.builder.appName(f'Anomaly Eval {mssparkutils.runtime.context}').getOrCreate()
config_path = f"abfss://configuration@{blob_account_name}.dfs.{azure_storage_domain}/config.global.json"
config = json.loads(''.join(sc.textFile(config_path).collect()))
teams_webhook_endpoint = config['rule_sets']['teams_webhook_uri']
alert_email = config['rule_sets']['alert_email']
web_app_uri = config['rule_sets']['webapp_uri']


# Setup Event Grid client 
subscription_id = TokenLibrary.getSecretWithLS("keyvault", 'SubscriptionId')
resource_group_name = TokenLibrary.getSecretWithLS("keyvault", 'ResourceGroupName')
event_grid_topic_name = TokenLibrary.getSecretWithLS("keyvault", 'EventGridTopicName')
event_grid_topic_endpoint = TokenLibrary.getSecretWithLS("keyvault", 'EventGridTopicEndpointUri')
tenant_id = TokenLibrary.getSecretWithLS("keyvault", 'TenantID')
client_id = TokenLibrary.getSecretWithLS("keyvault", 'ADAppRegClientId')
client_secret = TokenLibrary.getSecretWithLS("keyvault", 'ADAppRegClientSecret')
event_grid_topic = f'/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.EventGrid/topics/{event_grid_topic_name}'
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
client = EventGridPublisherClient(event_grid_topic_endpoint, credential)

In [ ]:
# Send Event Grid Event 
def format_time(t): 
    if isinstance(t, str): 
        return t 
    if isinstance(t, (float,int)):
        return datetime.fromtimestamp(int(t)).isoformat()

    return t 

def now(): 
    return datetime.now().strftime("%Y-%m-%dT%H:%M:%S%Z")

def build_details_url(web_app_uri, params): 
    url_params = '&'.join([f'{k}={v}' for k,v in params.items()])
    return f'https://{web_app_uri}/anomaly?{url_params}'

def format_location(location): 
    ul = ''
    lr = ''
    for corner in anomaly_location.split('\n'): 
        if corner.startswith('Upper Left'): 
            ul = corner 
        if corner.startswith('Lower Right'): 
            lr = corner 

    upper_left = ul.split(')')[0].replace('(', '').replace('Upper Left', '').replace(' ', '')
    lower_right = lr.split(')')[0].replace('(', '').replace('Lower Right', '').replace(' ', '')
    return f'{upper_left} X {lower_right}'



if anomaly_found: 
    with tracer.span('Send anomaly alert to event grid'):
        location = format_location(anomaly_location)
        params = {
            'date': anomaly_time,
            'location': location,
            'source_image': input_image_low_res,
            'source_image_high_res': input_image,
            'ship_bb_image': ship_bb_image_high_res,
            'ship_bb_image_high_res': ship_bb_image_high_res,
            'ais_image': ais_image,
            'anomaly_image': anomaly_image
        }
        details_url = build_details_url(web_app_uri, params)       
        event_data = {
            'eventDate': now(), 
            'eventMetrics':{
                'ais_csv_file': input_ais_csv_file,
                'input_image': input_image,
                'anomaly_location': location,
                'anomaly_time': format_time(anomaly_time),
                'event_detail_uri': details_url
            },
            'teams_webhook_endpoint': teams_webhook_endpoint,
            'alert_email': alert_email

        }
        event = EventGridEvent(data = event_data, subject="MINTED/AnomalyAlert", event_type="MINTED.ruleTriggered", data_version="1.0", topic=event_grid_topic)
        from pprint import pprint 
        pprint(event_data)
        client.send(event)